In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = [10, 5]


In [2]:
def calStability(raw, task_order, method):
    values = {k:[] for k in method}

    for t in task_order:
        rows = raw[raw["task_order"]==t]
        offline = rows[rows["method"]=="offline"]
        
        for m in method:                
            target = rows[rows["method"]==m]
            _m = target[target["task_index"]==1][["accuracy", "no_of_test", "no_of_correct_prediction"]]
            
            if m=="offline":
                continue
                
            
            _ideal = offline[offline["task_index"]==1]["accuracy"]
            val = float((_m["accuracy"]/float(_ideal)).sum()/len(_m))
            values[m].append(val)
    
    return values

In [3]:
def calPlasticity(raw, task_order, method):
    values = {k:[] for k in method}

    for t in task_order:
        rows = raw[raw["task_order"]==t]
        offline = rows[rows["method"]=="offline"]
        
        for m in method:
            _sum = 0.0
                
            target = rows[rows["method"]==m]
            train_session = target["train_session"].unique()
            
            if m=="offline":
                continue
    
            _m = target[target["train_session"]==(target["task_index"])][["accuracy", "no_of_test", "no_of_correct_prediction"]]
            
            _ideal = offline["accuracy"]
            val = _m["accuracy"].div(_ideal.values, axis=0).sum()/len(_m)
            values[m].append(val)
            
    
    return values

In [4]:
def calOverallAcc(raw, task_order, method):
    values = {k:[] for k in method}

    for t in task_order:
        rows = raw[raw["task_order"]==t]
        offline = rows[rows["method"]=="offline"]
        
        for m in method:
            if m=="offline":
                continue
                    
            _sum = 0.0
                
            target = rows[rows["method"]==m]
            
            task_index = target["task_index"].unique()
            train_session = target["train_session"].unique()
            
            _m = target[target["train_session"]==len(task_index)][["accuracy", "no_of_test", "no_of_correct_prediction"]]
            
            _ideal = offline["accuracy"]
            val = _m["accuracy"].div(_ideal.values, axis=0).sum()/len(_m)
            values[m].append(val)
            
    return values

In [9]:
all_values = {}
for d in ["HouseA", "CASAS", "PAMAP", "DSADS"]:
    dataset = d
    folder = "../../Results/"+dataset+"/exp_no_of_hidden/"
    
    raw = pd.read_csv(folder+"results.txt")
    raw.columns = [c.strip() for c in raw.columns]
    raw["train_session"] = pd.to_numeric(raw["train_session"], errors='coerce')
    
    cmd = raw["cmd"].unique()
    task_order = raw["task_order"].unique()
    method = raw["method"].unique()

    stability = []
    plasticity = []
    overallAcc = []

    for c in cmd:
       
        target = raw[raw["cmd"]==c]
        m = calStability(target, task_order, method)
        stability.append(m)
        
        m = calPlasticity(target, task_order, method)
        plasticity.append(m)
        
        m = calOverallAcc(target, task_order, method)
        overallAcc.append(m)
        
    all_values[d] = (stability, plasticity, overallAcc)
    print(d, "DONE")

HouseA DONE
CASAS DONE
PAMAP DONE
DSADS DONE


# Continual learning 

In [17]:
from scipy import stats

def selectModel(db):
    return all_values[db]

def printCLMetrics(values, cmd):
    
    (stability, plasticity, overallAcc) = values
    stability  = stability[cmd]
    plasticity = plasticity[cmd]
    overallAcc = overallAcc[cmd]
    
    def p(metric, name):
        print("Metric: ", name)
        for m in metric:
            avg = np.mean(metric[m])
            err = stats.sem(metric[m])
            print("{0} {1:.3f} {2:.3f}".format(m, avg, err))
        print("=====================")
        print("")
        
    p(stability, "M base")
    p(plasticity, "M new")
    p(overallAcc, "M overall")

In [18]:
m = selectModel("HouseA")
printCLMetrics(m, 3)

Metric:  M base
offline nan nan
none 0.358 0.007
exact 1.027 0.010
mp-gan 0.929 0.032
mp-wgan 0.870 0.050
sg-cgan 0.961 0.022
sg-cwgan 0.818 0.037
lwf 0.510 0.056
ewc 0.378 0.021

Metric:  M new
offline nan nan
none 1.126 0.007
exact 1.067 0.007
mp-gan 1.017 0.020
mp-wgan 1.034 0.022
sg-cgan 1.021 0.024
sg-cwgan 1.063 0.016
lwf 1.074 0.021
ewc 1.126 0.007

Metric:  M overall
offline nan nan
none 0.388 0.007
exact 1.006 0.006
mp-gan 0.843 0.028
mp-wgan 0.825 0.036
sg-cgan 0.888 0.025
sg-cwgan 0.840 0.030
lwf 0.535 0.040
ewc 0.385 0.008



In [19]:
m = selectModel("CASAS")
printCLMetrics(m, 3)

Metric:  M base
offline nan nan
none 0.286 0.050
exact 1.226 0.164
mp-gan 0.994 0.157
mp-wgan 0.899 0.166
sg-cgan 1.145 0.252
sg-cwgan 0.877 0.101
lwf 0.468 0.116
ewc 0.283 0.049

Metric:  M new
offline nan nan
none 1.195 0.048
exact 1.173 0.049
mp-gan 1.069 0.067
mp-wgan 1.120 0.065
sg-cgan 0.986 0.070
sg-cwgan 1.134 0.068
lwf 1.143 0.069
ewc 1.203 0.046

Metric:  M overall
offline nan nan
none 0.231 0.023
exact 1.037 0.038
mp-gan 0.801 0.043
mp-wgan 0.798 0.035
sg-cgan 0.827 0.063
sg-cwgan 0.758 0.038
lwf 0.498 0.043
ewc 0.229 0.023



In [20]:
m = selectModel("PAMAP")
printCLMetrics(m, 3)

Metric:  M base
offline nan nan
none 0.269 0.020
exact 1.116 0.054
mp-gan 0.608 0.046
mp-wgan 0.298 0.039
sg-cgan 0.793 0.059
sg-cwgan 0.509 0.061
lwf 0.269 0.020
ewc 0.226 0.011

Metric:  M new
offline nan nan
none 1.176 0.034
exact 0.977 0.055
mp-gan 1.153 0.029
mp-wgan 1.167 0.032
sg-cgan 1.138 0.038
sg-cwgan 1.161 0.033
lwf 1.176 0.034
ewc 1.196 0.035

Metric:  M overall
offline nan nan
none 0.319 0.023
exact 0.972 0.056
mp-gan 0.682 0.042
mp-wgan 0.418 0.031
sg-cgan 0.848 0.042
sg-cwgan 0.485 0.038
lwf 0.319 0.023
ewc 0.221 0.005



In [21]:
m = selectModel("DSADS")
printCLMetrics(m, 3)

Metric:  M base
offline nan nan
none 0.420 0.059
exact 1.972 0.306
mp-gan 0.640 0.076
mp-wgan 0.448 0.061
sg-cgan 0.972 0.227
sg-cwgan 0.566 0.078
lwf 0.420 0.059
ewc 0.396 0.062

Metric:  M new
offline nan nan
none 1.440 0.137
exact 0.848 0.110
mp-gan 1.334 0.113
mp-wgan 1.463 0.132
sg-cgan 1.336 0.107
sg-cwgan 1.447 0.137
lwf 1.440 0.137
ewc 1.612 0.102

Metric:  M overall
offline nan nan
none 0.327 0.045
exact 0.833 0.112
mp-gan 0.660 0.056
mp-wgan 0.386 0.062
sg-cgan 0.767 0.086
sg-cwgan 0.422 0.059
lwf 0.327 0.045
ewc 0.277 0.027

